# Exploring PyTorch and Transformers for Genomic Sequence Classification

## Outline:
1. Objectives
2. What is *deep learning*?
3. Some cases of deep learning in genomics
4. Deep learning implementation workflow
    * Dataset
    * Model training with HuggingFace's `transformers`
        * Model selection
        * Tokenization
        * Training
        * Evaluation
    * Train a PyTorch model with your own data
        * Dataset
            * The `Dataset` and `DataLoader` objects
        * Tokenization
        * Model building
        * Training
        * Evaluation
5. Conclusion
6. Contact

# Objectives

1. Learn the basic theory and practice of deep learning
2. Understand the basic deep learning workflow
3. Deploy a model for genomic sequence classification
    * With helpers
    * Manually

# What is *Deep Learning*?
Deep Learning is a subset of machine learning techniques that uses **artificial neural network**-based models (ANN). What makes it **deep** is the presence of many transformation *layers* within the models. Figure 1 shows a Venn diagram of the organization between artificial intelligence, machine and deep learning. 
Deep learning models are able to learn from raw data. This is one of the main differences against traditional learning pipelines. With these kind of models you can assemble a learning system that tunes itself automatically rather than fixing and updating each individial component one by one. It replaces some of the labor-intensive processes needed for other methods, like field-specific data preprocessing and/or manual feature extraction. Deep learning models can learn and process these features in an automated fashion, generate accurate predictions, and be fine-tuned for specific applications when an available model exist [1].

One of the most common and persistent disadvantages of applying deep learning methods in your work is the large amount of data needed to train the model. To capture the features and generalize the phenomena in your studies, a substantial amount of data (sometimes labeled) must be available so the model can fit them and produce accurate predictions. However, with the ever-increasing availability of graphical processing units (GPUs), the massive amounts of data generated in clinical and biological scenarios, and the possibility of fine-tuning existing models, implementing a deep learning architecture for your specific applications is becoming increasingly simple [1].

<fig>
<img    src="images/ENG_IA_ML_DL.png"
        width=600
        height=600>
<figcaption>Fig. 1: Venn diagram displaying the organization between AI, machine, and deep learning.
</figure>

# Some cases of study of deep learning in genomics

## Enformer
Avsec and collaborators [2] developed a hybrid deep learning architecture that combines two powerful components: convolutional and attention layers. The authors highlight three main advantages of their model over other prediction methods.
1. The model can accurately predict activating or repressive mutations affecting gene expression.
2. Its predictions are not limited to conserved enhancers, allowing for individual-specific gene expression predictions.
3. It produces predictions independently from the genomic sequence type, enabling the design and assessment of synthetic and specific sequences.

<fig>
<img    src="images/Enformer.png"
        width=800
        height=300>
<figcaption>Fig. 2: Enformer article [2].
</figure>

## HyenaDNA
HyenaDNA [3] is a recent model with state-of-the-art performance in genomic sequence classification and next-token prediction tasks. Nguyen and collaborators replaced the conventional attention layer in transformers with the Hyena block, an operator consisting of long convolutions and fully connected layers that captures longer contexts while maintaining single-nucleotide resolution. They evaluated the model’s performance on the Genomic Benchmarks datasets [4] and compared it against other leading deep learning models. HyenaDNA is the first genomic model capable of processing inputs of up to 1 million tokens (nucleotides).

<fig>
<img    src="images/HyenaDNA.png"
        width=800
        height=300>
<figcaption>Fig. 3: HyenaDNA article [3].
</figure>

## DeepRVAT
Clarke and collaborators created a model based on deep set neural networks for gene discovery and trait prediction based on gene impairment scores and rare variant annotations using whole-exome-sequencing data. The model, named *DeepRVAT*, performs association tests on rare variants in a data-driven way exclusively, and can perform these tests on a biobank level. On top of the model's development, they identified new rare-variant associations to several phenotypes [5].

<fig>
<img    src="images/DeepRVAT_article.png"
        width=800
        height=400>
<figcaption>Fig. 4: DeepRVAT article [5].
</figure>

# Deep Learning implementation workflow
Several advances and discoveries in biological and other *omics* fields in recent years have been driven by novel machine and deep learning methods. These techniques have made significant progress in applications ranging from computer vision to natural language processing [6]. Their popularity can be attributed to the relatively recent increase in computational power and accessibility. However, their theory and application can sometimes be complex and confusing. Given their growing use, there are now many programming tools that simplify their implementation. This process generally involves four key steps:

1.	Select and curate data.
2.	Choose a model.
3.	Evaluate the model’s performance.
4.	Interpret the results.

This tutorial will guide you through this process and show you how to apply these techniques for your own experiments.


## Import modules and libraries
Importing a module means you're going to use other people's code and functions for your algorithm.

In [ ]:
! pip install genomic_benchmarks datasets evaluate --quiet

In [32]:
# This cell of code is used to import the necessary libraries for the notebook
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import torch.nn.functional as F

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split

import gdown

from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import load_dataset, Dataset
import evaluate
from tqdm.auto import tqdm

from genomic_benchmarks.data_check import list_datasets, info, is_downloaded
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.dataset_getters.pytorch_datasets import HumanEnhancersCohn

## Dataset

We will be working with the **Genomic Benchmarks** datasets, a set of benchmarks for classification of genomic sequences to test models' capabilities. All the datasets are described in [[4]](https://doi.org/10.1186/s12863-023-01123-8). Grešová and their collaborators created this python module for easy access to well-labeled and balanced datasets as well as to enable reproducibility for genomic-related deep learning models.

In the next code cell we list the available datasets in the **Genomic Benchmarks** module:

In [ ]:
list_datasets()

For illustrative pourpuses you will work on the `human_enhancers_cohn` dataset which contains multiple genomic sequences that **are** or **are not** enhancers. In machine learning verbose, you will work on a **binary classification** problem.
### What is an enhancer?

An **enhancer** is a genomic regulatory element that influences gene transcription. It can be located quite far from the gene it regulates, sometimes up to 1 million base pairs away. Although enhancers don't produce any direct products, they play a crucial role in regulating gene expression. This is significant because many diseases are caused or associated with deficiencies in specific genes. 

You may have encountered a similar term, which is **promoter**. The difference between promoters and enhancers is their proximity to the gene they regulate. Promoters are close to the gene they affect. Their functionality are pretty the same, cause the affected gene to over-express and generate more products or proteins than it would normally do.

<fig>
<img    src="images/enhancer_promoter.png"
        width=1000
        height=220>
<figcaption>Fig. 5: Graphical representation of the distance of an enhancer and a promoter from a gene.
</figure>

Run the next code cell to show basic information of the dataset that was selected.

In [ ]:
info("human_enhancers_cohn", version=0)

The `genomic_benchmarks` module offers multiple data handlers and helpers to load, show and give you an idea of how each of its datasets are composed.
In the next code cell we use the `HumanEnhancersCohn` function to download and assign the dataset into two variables, `train_dataset` and `test_dataset`, respectively.

In [4]:
# Load the dataset and split it into training and test sets
train_dataset = HumanEnhancersCohn(split="train", version=0)
test_dataset = HumanEnhancersCohn(split="test", version=0)

Just to make sure we imported the correct dataset we can print the lengths of each set and check if the numbers match the ones shown above.

In [ ]:
# Print the lengths of each set check if they match the info we saw previously
print(f"Length of the training dataset: {len(train_dataset)}. Length of the test dataset: {len(test_dataset)}") 

But how this data actually look like? You have only downloaded some data from a library for binary classification up to this point. How can you actually see some samples? Turns out to be very easy to do so! Let's see two examples, one `positive` and one `negative` sample.

In [ ]:
# Get a sample from train_dataset with a 1 on the second element of the tuple
positive_sample = next(filter(lambda x: x[1] == 1, train_dataset)) # A 1 indicates a positive sample
negative_sample = next(filter(lambda x: x[1] == 0, train_dataset)) # A 0 indicates a negative sample
print(f"{positive_sample}\n{negative_sample}")

Can you tell, without looking at the corresponding labels, which of these sequences is an enhancer and which is not?
Well, you can determine this by some other studies like genome-wide chromatin immunoprecipitation or RNA sequencing. But if you saw these sequences alone, would you be able to tell whether it is an enhacer or not? A deep learning model can do it! At the cost of massive amounts of already-labeled data that were obtained with techniques like the previously mentioned (that is, what we just downloaded a couple of cells above).

## Model training with HuggingFace
HuggingFace's `transformers` library, introduced in 2020, is an open-source library aimed at making powerful deep learning architectures, such as transformers, easily accessible [7]. It was designed for robust, reliable, and simple implementation of the Natural Language Processing (NLP) workflows, including data processing, model selection, and prediction generation. They also created specific modules and libraries for dataset and tokenizer retrieval which, you will be using. 

The basic steps to train (or fine-tune) a `transformers` model are:
1. Pick a model from the HuggingFace hub 
2. Select the tokenizer that corresponds to the model you picked
3. Apply the tokenizer to the sequences in your dataset
4. Define a training configuration
5. Set up the Trainer
6. Train the model
7. Evaluate the model

More information on the [HuggingFace website](https://huggingface.co).

### Model selection

<fig>
<img    src="images/deeplearning_implementation_workflow.png"
        width=1000
        height=290>
<figcaption>Fig. 6: HuggingFace's models implementation workflow. Model selection.
</figure>


HuggingFace offers a wide range of pre-trained models. You can select a model based on your specific task, such as text classification or text generation. You’re going to work with the HyenaDNA model [3], a pre-trained model designed to predict the next token in a sequence, meaning it generates the next nucleotide in a genomic sequence based on the context of the preceding nucleotides. This model differs from conventional transformers by replacing the characteristic attention layer with a **long convolution**, giving it the ability to capture a wider context at each layer while maintaining single-nucleotide resolution. The smallest instance of the model can be accessed by using the string `LongSafari/hyenadna-tiny-1k-seqlen-hf` as an argument for the `AutoModelForSequenceClassification` function, as shown below.

In [ ]:
# Define the model instance using the checkpoint we want to use
checkpoint = 'LongSafari/hyenadna-tiny-1k-seqlen-hf' # This is the model's name we are going to use
# The num_labels parameter is set to 2 because we have two classes in the dataset (positive and negative)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, trust_remote_code=True, labels=2, safe_serialization=False)

### Tokenization

<fig>
<img    src="images/deeplearning_implementation_workflow-2.png"
        width=1000
        height=250>
<figcaption>Fig. 7: HuggingFace's models implementation workflow. Tokenization.
</figure>

After selecting the model, you need to choose the corresponding tokenizer to ensure compatibility with the model’s inputs. The tokenizer will encode, pad, and truncate the sequences as necessary, converting them into input tensors that can be used for training or fine-tuning. Formally, **tokenization** is the process of encoding words and converting them into meaningful numerical representations, which is a key step in the **Natural Language Processing** (NLP) workflow. NLP is a subfield of computer science and AI that uses various algorithms to enable computers to understand human language. Its applications range from text encoding and generation to voice recognition and speech systems. Examples of NLP in daily life include chatbots and digital assistants such as Alexa, Bixby, Siri, and Google Assistant.

In this case, you’re working on sequence classification, meaning each of these sequences has an inherent “grammar” or structure that defines them as enhancers or not. Just as NLP analyzes sentence grammar by encoding each word, you’ll do the same with genomic sequences.

Today, implementing and using a tokenizer to convert  raw genomic sequences into numerical representations is straightforward, thanks to Hugging Face’s `transformers` and `tokenizers` libraries.

In [8]:
# Define a max_length variable to be used in the tokenizer
max_length = 1024 # This variable will represent the maximum length of the input sequences

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

### Apply the tokenizer to the sequences in your dataset and prepare the dataset
Fortunately, the same authors of the `Genomic_Benchmarks` datasets made them available in the Hugging Face hub. All you need to do is retrive the dataset with the `load_dataset` function. The dataset's information is available at the HuggingFace Hub in [this link](https://huggingface.co/datasets/katarinagresova/Genomic_Benchmarks_human_enhancers_cohn).

In [10]:
# Retrieve the dataset from the HF hub
datataset_card = "katarinagresova/Genomic_Benchmarks_human_enhancers_cohn"
raw_datasets = load_dataset(datataset_card)

Let's take a look to the `raw_datasets` object:

In [ ]:
# Print the dataset's details
print(raw_datasets)

As you can see, it is already splitted in `train` and `test` sets. Let's take a look to the first sample in the `train` set. It should be pretty the same to those you've already seen above.

In [ ]:
# Print the first element of the training set to see the structure of the dataset
print(raw_datasets['train'][0])
print(f"The sample has the following keys: {raw_datasets['train'][0].keys()}")

Each sample in both the `train` and `test` sets is a dictionary object containing the `seq` and `label` keys, representing the raw sequence and its corresponding label, which indicates whether it is an enhancer or not.

Now you have to apply the tokenizer only to the sequences in the dataset (not the labels), this will be done by using an auxiliar function called `tokenize_function`.

In [14]:
# Define a function to tokenize the sequences in the dataset
def tokenize_function(example):
    return tokenizer(example["seq"], truncation=True)

# Tokenize the dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # Define the data collator to pad the sequences to the same length in a batch

### Use the `Trainer` API

<fig>
<img    src="images/deeplearning_implementation_workflow-3.png"
        width=1000
        height=270>
<figcaption>Fig. 8: HuggingFace's models implementation workflow. Model and training configuration.
</figure>

#### Configure the training parameters
The `transformers` module includes a helper that allows you to train a model by simply specifying the training hyperparameters like learning rate, number of epochs and evaluation metrics, making the training and fine-tuning processes straightforward. You will also calculate the model's accuracy after each epoch on the test set. To do this you can see in the next code cell there's a function named `compute_metrics` which will be included in the `TrainingArguments` object. Using the `evaluate` module calculating any evaluation metric is straighforward and easily implemented. You can check all the available evaluation metrics HuggingFace offers at [this website](https://huggingface.co/evaluate-metric).

In [ ]:
# Declare an auxiliar function to calculate evaluation metrics
def compute_metrics(eval_preds):
    metric = evaluate.load("accuracy")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define the training arguments for the model
epochs = 5
training_args = TrainingArguments("test-model", eval_strategy="epoch", num_train_epochs=epochs, save_safetensors=False)

#### Set up the `Trainer` object
Hugging Face's `Trainer` class simplifies the training and evaluation loops by handling the optimizer and loss function automatically. All you need to do is pass the mode, training arguments, and dataset to it.

In [ ]:
# Build the trainer instance
trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

#### Train the model
Everything is ready to train the model now! All you need to do is execute the `trainer.train()` method and you will see the training progress.

<fig>
<img    src="images/deeplearning_implementation_workflow-4.png"
        width=1000
        height=240>
<figcaption>Fig. 9: HuggingFace's models implementation workflow. Model's training phase.
</figure>

In [ ]:
# Train the model
trainer.train()

Excellent! You just trained a deep learning model that classifies between enhancers and random genomic sequences. But the next question is: how well the model is performing? Or how reliable are its predictions? Is there any measure that can tell you this? The answer is **yes**, there exist many **evaluation metrics** that inform the model's performance. One of the most used ones is **accuracy** which tells you how many **correct** predictions yielded out of the total number of predictions.

## Evaluation

<fig>
<img    src="images/deeplearning_implementation_workflow-5.png"
        width=1000
        height=240>
<figcaption>Fig. 10: HuggingFace's models implementation workflow. Evaluation to assess model's performance.
</figure>

Excellent! You just finished training your model, now what? Is it ready to predict which sequence is an enhancer and which is not? As you may saw during the training process at the end of each epoch some values were displayed. These inform you about the model's learning process. You can plot these values and watch the model's evolution through the epochs.

In [ ]:
# Extract the metrics of the evaluation loops in each epoch
eval_metrics = [item for item in trainer.state.log_history if "eval_loss" in item]
print(eval_metrics)

In [18]:
# Extract the loss and accuracies in differente lists
losses = [item['eval_loss'] for item in eval_metrics]
accuracies = [item["eval_accuracy"] for item in eval_metrics]

### Accuracy and Loss

The **loss value** is a metric that indicates how far the model’s predictions are from the ground truth—the lower the value, the better the model is performing.

In contrast, **accuracy** represents the percentage of correct predictions out of the total number of predictions. In other words, it shows how often the model correctly predicts either the positive or negative classes.

The following figure illustrates the ideal relationship between these two metrics:

<fig>
<img    src="images/loss_and_accuracy.png"
        width=1000
        height=600>
<figcaption>Fig. 11: The ideal progression of the loss value and accuracy metrics: A decreasing loss value indicates that the model is making increasingly better predictions. For accuracy, a higher value means the model is generating more correct predictions.
</figure>

In [ ]:
# Plot the loss and accuracy values agianst the number of epochs
n_epochs = np.arange(1, epochs + 1) # Create an array with the number of epochs
plt.figure(figsize=(18, 6))
plt.subplot(1, 2, 1)
plt.plot(n_epochs, losses)
plt.xticks(n_epochs)
plt.title("Loss vs. Number of epochs")
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.subplot(1, 2, 2)
plt.plot(n_epochs, accuracies)
plt.xticks(n_epochs)
plt.title("Accuracy vs. Number of epochs")
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.show()

### Confusion matrix

Another way of assessing how well your model works is with a **confusion matrix**. This is a metric that quantifies and shows the number of correct predictions against the incorrect ones. It compares the number of predicted values against the true ones, allowing to check how often the model misses to clasify correctly a sample.

To calculate it you just need to generate predictions with your model and compare them against the labels contained in the dataset. In the next code cell it's showed how to do this.

The following figure explains how to interpret a confusion matrix:

<fig>
<img    src="images/ENG_Confusion_matrix.png"
        width=700
        height=600>
<figcaption>Fig. 12: The confusion matrix explained.
</figure>

In [ ]:
# We need to produce a set of predictions on the test set
model_out = trainer.predict(tokenized_datasets["test"])
logits = model_out.predictions
predictions = np.argmax(logits, axis=-1)

# Load the confusion matrix metric from HF
confusion_matrix_metric = evaluate.load("confusion_matrix")
# Compute the confusion matrix
confusion_matrix = confusion_matrix_metric.compute(predictions=predictions, references=model_out.label_ids)

# Display the confusion matrix
plt.figure(figsize=(8, 6)) 
sns.heatmap(confusion_matrix['confusion_matrix'], annot=True, fmt='d', cmap=sns.light_palette("seagreen", as_cmap=True)) # Create a heatmap of the confusion matrix
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

### Receiver-Operator Characteristic (ROC) curve
The **ROC curve** plots the true positive rate (TPR) against the false positive rate (FPR). While it’s typically used for binary classification tasks, it can also be adapted for multi-class classification with some adjustments. An ideal ROC curve would rise sharply from the origin to the top-left corner and then extend horizontally to the top-right corner. The greater the area under the curve (AUC), the better the classifier’s performance, with a full AUC representing a perfect classifier.

The following image shows how a ROC curve shoulb be interpreted.

<fig>
<img    src="images/ROC_curve.png"
        width=700
        height=600>
<figcaption>Fig. 13: A receiver-operator characteristic curve (ROC).
</figure>


In the next code cell, it is shown how to generate a ROC curve using the same `model_out.label_ids` and `predictions` variables as in the confusion matrix.

In [ ]:
# Plot a ROC curve 
roc_auc = roc_auc_score(model_out.label_ids, predictions) # Calculate the ROC AUC score
fpr, tpr, _ = roc_curve(model_out.label_ids, predictions) # Calculate the false positive rate and true positive rate

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') # Plot the ROC curve
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

### Summary on these and more evaluation metrics
Evaluation metrics are really important because they inform you the model's performance, giving you an idea if your model is capable of solving the problem you're attacking, or not. Here you can find 3 more metrics, but there exist many more. Erickson and Kitamura [8] decribe them more in depth on their paper.

Abbreviations:
* $TP$: True Positives
* $TN$: True Negatives
* $FP$: False Positives
* $FN$: False Negatives

#### Accuracy
It's the quantity of correct predictions out of the total number of samples within a dataset. Its value is commonly represented from 0% to 100%.

$$Accuracy = \frac{TP + TN}{TP + FN + TN + FP}$$

#### Sensitivity 
It is the true positive rate of the model's test. In other words, means how many of the enhancer sequences were actually categorized as enhancers.
Recall the confusion matrix where 2,465 out of 3,474 sequences were correctly classified (results when preparing the workshop). Its value goes from 0 to 100% as well.

$$Sensitivity = \frac{TP}{TP+FN}$$

#### Specificity
This is the true negative rate when testing a predictive model. In this case it represents the proportion of the sequences that **are not** enhancers and were correctly differentiated from the enhancer sequences. Again, if you see the confusion matrix you can see in the first row that 2,438 sequences out of 3474 were correctly classified as **not enhacers**.

$$Specificity = \frac{TN}{TN+FP}$$

#### Precision
This one represents the truly positive sequences from all the samples that were classified as *positive* by the model. In this case only the second column in the confusion matrix is considered. Its values goes from 0 to 100% too. 

$$Precision = \frac{TP}{TP+FP}$$

#### The receiver-operator characteristic curve (ROC)
<fig>
<img    src="images/ROC_curve.png"
        width=700
        height=600>
</figure>

#### Confusion matrix

<fig>
<img    src="images/ENG_Confusion_matrix.png"
        width=700
        height=600>
</figure>

---

# Training a PyTorch model with your own data

You have successfully trained a deep learning model using a dataset that contains genomic sequences that either **are** or **are not** enhancers. But what if the data for your experiment is different? Can you still train the same model? How would you pre-process your data to make it compatible with these neural networks? Also, what if you want to build your own model or tweak it however you want?

To finish this workshop, you will download a new dataset in `csv` format containing **coding** and **non-coding** sequences (extracted from [9]), transform it into representable tensors, build and train a new model, and evaluate its performance.

## Prepare the data
The data you're about to download is a `csv` file with genomic sequences representing coding **or** non-coding sequences. This data is located in a drive folder, you can download and give a check to the data with the following code cell.

In [ ]:
# Step 1: Get the URL of the CSV file
file_id = "1itLZCkFbeqlscbrrZdhX-9qA5j2Hc_W_"
url = f"https://drive.google.com/uc?id={file_id}"  # Replace with the actual URL of the CSV file

# Step 2: Download the file using gdown
output = 'data.csv'  # The name of the downloaded file
gdown.download(url, output, quiet=True)

# Step 3: Load the CSV file into a DataFrame
df = pd.read_csv(output)

# Display the first few rows of the DataFrame
df.head()

You have already downloaded the dataset, but it isn't ready to be used with a neural network yet. PyTorch neural networks expect their inputs to be arranged in a special data structure known as **tensor**. If you have experience with Numpy's `ndarray`s getting to know tensors will be pretty easy. These are $n$-dimensional number arrays optimized for gradient calculus and other operations that run in the background when training a neural network. More information about tensors and their specifications on [PyTorch's website](https://pytorch.org).

But how does a tensor look like? In the following code cells we initialize random tensors and display them just for illustrative purposes.

In [ ]:
my_tensor = torch.tensor([1, 2, 3, 4, 5]) # As you can see, it takes a list an input
print(my_tensor)

Tensor's data type is an important consideration always. Many errors arise when tensor's data types aren't the same bewteen the inputs and the labels, and they ensure compatibility with the model up to some extent. Specifically, this error arises when calculating the **loss**, a value that measures the difference between the model predictions and the ground-truth (your labels), or when propagating the information through the model.

In [ ]:
my_tensor.dtype # Using the dtype attribute, we can see the data type of the tensor

You can manipulate tensors by adding or removing elements, changing their data types, do any mathematical or arithmetical operation with them, etc.

In [ ]:
my_tensor = my_tensor.to(torch.float64) # We can change the data type of the tensor using the .float() method
print(my_tensor.dtype)

In [ ]:
tensor_mul = my_tensor * my_tensor # We can perform element-wise multiplication on tensors
print(tensor_mul)

## Tokenization
Now that you know you need these sequences in a specific format, the next question is: how do you transform these sequences into representable tensors? Here's where the tokenizer makes its entrance again! You can either create your own tokenizer or use any tokenizer available in the HuggingFace Hub. For the pourpuses of this tutorial you will use the HyenaDNA tokenizer once again.

In [35]:
# instantiate tokenizer
checkpoint = 'LongSafari/hyenadna-tiny-1k-seqlen-hf' # This is the model's name we are going to use
max_length = 1024 # This variable will represent the maximum length of the input sequences

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

Let's apply this tokenizer to the first sequence to look how the data comes out of it.

In [ ]:
enconded_sequence = tokenizer(df["sequences"][0], truncation=True, padding="max_length", max_length=max_length)
print(enconded_sequence)

As you can see, now your sequence is represented as a bunch of numbers. Each of them represents each of the nucleotides that conform the sequence you used as input and the tokenizer adds some special tokens to differentiate from sequence to sequence when passed into the training phase. Let's see how the network will read your sequences in meaningful way for you.

In [ ]:
decoded = tokenizer.decode(enconded_sequence['input_ids'])
print(decoded)
print(f"The special token the tokenizer adds is: {tokenizer.decode(enconded_sequence['input_ids'][-1])}")

If you want to obtain the PyTorch tensors directly just add the `return_tensors="pt"` parameter.

In [ ]:
encoded_tensors = tokenizer(df["sequences"][0], truncation=True, padding=True, max_length=max_length, return_tensors='pt')
print(encoded_tensors)

The output's data structure is a dictionary containing the `input_ids` key which contains the actual tensors you're going to use to train your model. There's still one problem, we need to pass these encoded sequences to the model in **mini-batches**, we have just converted only one sequence up to this point. How can you convert and tokenize these sequences in mini-batches? 

### The `Dataset` and `DataLoader` objects
To make this data set compatible with PyTorch and HuggingFace models you need to turn it into a `Dataset` object. As you already have many variables containing the information about the model, the tokenizer, the loss function and many others, training again the model will be pretty straightforward.

To create a `Dataset` object run the next code cell:

In [36]:
# Class for ncRNA and cDNA Dataset object
# It is important to note that in Jupyter notebooks, the class definition should be in the same cell as the import statements
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

class coding_and_noncoding_dataset(Dataset):
    '''
    Class for the coding and noncoding dataset from Ensembl.
    The dataset is a CSV file with two columns: "sequences" and "value". Being 0 for noncoding and 1 for coding.
    You can split the dataset into train and test sets by setting split=True and defining the test_size.
    For reproducibility, you can set the random_seed (defaulted to 42).
    '''
    def __init__( # Here are defined all the properties of the new object
        self,
        data_path,
        max_length,
        tokenizer = None,
        split='train',
        test_size=0.2,
        random_seed=42,
        use_padding=None,
    ):
        # Here the properties are initialized
        self.data = pd.read_csv(data_path)
        self.max_length = max_length
        self.tokenizer = tokenizer
        self.split = split
        self.test_size = test_size
        self.random_seed = random_seed
        self.use_padding = use_padding

        # Split the dataset into train and test sets depending on the split parameter
        if self.split == 'train':
            self.seq, _, self.label, _ = train_test_split(self.data['sequences'], self.data['value'],
                                                          test_size=self.test_size, random_state=self.random_seed)
        elif self.split == 'test':
            _, self.seq, _, self.label = train_test_split(self.data['sequences'], self.data['value'],
                                                          test_size=self.test_size, random_state=self.random_seed)

    def __len__(self):
        # Return the length of the dataset
        return len(self.seq)

    def __getitem__(self, idx):
        # Define how the samples are going to be retrieved from the raw dataset
        x = self.seq.iloc[idx]
        y = self.label.iloc[idx]

        # Tokenize the sequence and get the input_ids
        seq = self.tokenizer(x,
            add_special_tokens=False,
            padding="max_length" if self.use_padding else None,
            max_length=self.max_length,
            truncation=True,
        )
        seq = seq["input_ids"]  # get input_ids

        # Convert into float tensors (y has to be wrapped in a list)
        seq = torch.tensor(seq).to(torch.float32)
        target = torch.tensor([y]).to(torch.float32)
        
        return seq, target

Now let's instantiate the `train` and `test` datasets:

In [37]:
# Path to the dataset you just downloaded
data_path = "./data.csv"
# Intantiate the train and test sets
train_set = coding_and_noncoding_dataset(data_path=data_path,
                                         split='train',
                                         tokenizer=tokenizer,
                                         use_padding=True,
                                         max_length=max_length)

test_set = coding_and_noncoding_dataset(data_path=data_path,
                                        split='test',
                                        tokenizer=tokenizer,
                                        use_padding=True,
                                        max_length=max_length)

In [38]:
# Define the batch size (number of samples the model will work with on each iteration)
batch_sz = 8
# Initialize the DataLoader objects. These help you to retrieve samples from the train and test sets
train_loader = DataLoader(train_set, batch_size=batch_sz, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_sz, shuffle=False)

In [ ]:
# Print a batch of samples to see how they look like
for batch, (x, y) in enumerate(train_loader):
    print(f"Batch number: {batch}\nInputs: {x}\nLabels: {y}")
    break

## Train the model
All you have to do is repeat the process you did some minutes ago! As it is already specified in the `coding_and_noncoding_dataset`, the sequences are already tokenized, all you have to do is a little adjustment in the train and test loops and you're good to go!

In [40]:
# Import the necessary libraries again as a new python class is defined
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class GenomicCNN(nn.Module):
    # Define the necessary properties for the model
    def __init__(self, input_size, in_channels, out_channels, conv_kernel_size, pool_kernel_size, conv_dilation, pool_dilation, conv_padding, pool_padding, conv_stride, pool_stride):
        super(GenomicCNN, self).__init__()

        # 1D Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels, out_channels, conv_kernel_size, padding=conv_padding, dilation=conv_dilation, stride=conv_stride)
        # Max Pooling layer
        self.pool = nn.MaxPool1d(pool_kernel_size, pool_stride, pool_padding, pool_dilation)
        # Flatten layer
        self.flatten = nn.Flatten(start_dim=1)
        # Calculation of flatten output based on the convolutional and pooling layers
        conv_output_length = ((input_size + 2 * conv_padding - conv_dilation * (conv_kernel_size-1) - 1)//conv_stride) + 1  # Output length after conv1
        pooled_output_length = ((conv_output_length + 2 * pool_padding - pool_dilation * (pool_kernel_size-1) - 1)//pool_stride) + 1 

        # Fully connected layers
        self.fc1 = nn.Linear(pooled_output_length * out_channels, 64)
        self.fc2 = nn.Linear(64, 1) # Output layer

    def forward(self, x):
        # Convolution + ReLU + Pooling
        x = self.pool(F.relu(self.conv1(x)))

        # Turn the output from the pool layer into a 1D tensor
        x = self.flatten(x)
        
        # Fully connected layer + ReLU
        x = F.relu(self.fc1(x))
        
        # Output layer (Sigmoid for binary classification)
        x = torch.sigmoid(self.fc2(x))
        
        return x

You are almost there to train your model! You're a few parameters and functions away of training the model. Let's finish the set up to start the training process!

To start its training a deep learning model needs:
* a `device`: the piece of hardware where all the math will happen. It's advised to use a GPU for accelerated training
* `loss function`: this tells you how much difference is there between the model's predictions and the ground-truth labels in the dataset
* a number of `epochs`: the amount of times the model will "see" the samples within the training dataset
* an `optimizer`: a way of calculating and update the model's parameters to fit better the data after each iteration
* a `training` and `test` loops: to declare how the information will pass through the network and in which moment to update the parameters

Let's declare them in the following code cell:

In [ ]:
# Define a device
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

# Define the model's parameters
input_size = max_length
in_channels = 1
out_channels = 64
conv_kernel_size = 128
pool_kernel_size = 64
conv_dilation = 1
pool_dilation = 1
conv_padding = 0
pool_padding = 0
conv_stride = 1
pool_stride = 1

model = GenomicCNN(input_size, in_channels, out_channels, conv_kernel_size, pool_kernel_size, conv_dilation,
                       pool_dilation, conv_padding, pool_padding, conv_stride, pool_stride).to(device)
print(model)

# Define the optimizer and the loss function
criterion = nn.BCELoss()

# create optimizer and define its parameters
learning_rate = 1e-5 # 0.00001
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)


In [64]:
# Define the training loop
def train(model, device, train_loader, criterion, optimizer):
    model.to(device) # Send the model to the training device (cpu, gpu, mps)
    model.train() # Set the model to training configuration
    size = len(train_loader.dataset) # Retrive the dataset's length
    for batch, (x, y) in enumerate(train_loader):
        x, y = x.to(device), y.to(device) # Move the data to the device
        x = x.unsqueeze(1) # Add an additional dimension to the input to ensure compatibility with CNNs
        # Forward pass
        outputs = model(x) # Get the outputs of the model
        # Backward pass
        loss = criterion(outputs, y) # Calculate the loss between the predictions and labels
        # Adjust model weights (parameters)
        loss.backward() 
        optimizer.step() 
        # Zero the gradients
        optimizer.zero_grad()
        if batch % 1000 == 0: # Every thousand iterations print the model status
            loss, current = loss.item(), (batch + 1) * len(x)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [65]:
# Define the test loop
def test(model, device, test_loader, losses, accuracies):
    model.to(device) # Move the model to the device
    model.eval() # Set the model to evaluation mode
    size = len(test_loader.dataset)
    test_loss, correct = 0, 0
    with torch.no_grad(): # Disable gradient calculation
        for x, y in test_loader:
            x, y = x.to(device), y.to(device) # Move the data to the device
            x = x.unsqueeze(1) # Add an additional dimension to the input to ensure compatibility with CNNs
            outputs = model(x) # Get the outputs of the model
            test_loss += criterion(outputs, y).item() # Get the loss value
            correct += (outputs.round() == y).float().mean() # Get the number of correct predictions
    test_loss /= size
    losses.append(test_loss) # Save the loss value in the current epoch
    correct /= size
    accuracies.append(correct) # Save the accuracy value in the current epoch
    print(f"Test Error: \n Accuracy: {(100*correct):>0.3f}%, Avg loss: {test_loss:>8f} \n")

Now you have all the parameters, objects, and functions needed to train a deep learning model! The last thing you have to do is to declare how many **epochs** the training phase will have. This is how many times will the model "see" or process all the samples contained in the training dataset. This is done to try to fit the model as most as possible to the data. That's why your dataset has to include a wide variety of examples, covering the most cases as possible from the phenomenon you're studying.

In [ ]:
# Define the number of epochs
epochs = 10

# Define the lists to store the losses and accuracies
losses = []
accuracies = []

# Initialize model's training
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}\n-------------------------------")
    train(model, device, train_loader, criterion, optimizer)
    test(model, device, test_loader, losses, accuracies)

## Evaluation

Excellent! You just finished training your model, now what? Is it ready to predict which sequence is an enhancer and which is not? As you may saw during the training process at the end of each epoch the accuracy and average loss values were displayed. These tell us how **reliable** our model is. You can plot these values and watch the model's evolution through the epochs because each time the `test` function was working it saved them.

### Accuracy and Loss

In [67]:
# Move the accuracy values to the cpu because they were calculated in the GPU or MPS device
accuracies = [acc.to("cpu") for acc in accuracies]

In [ ]:
# Plot the loss and accuracy values agianst the number of epochs
n_epochs = np.arange(1, epochs + 1) # Create an array with the number of epochs
plt.figure(figsize=(18, 6))
plt.subplot(1, 2, 1)
plt.plot(n_epochs, losses)
plt.xticks(n_epochs)
plt.title("Loss vs. Number of epochs")
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.subplot(1, 2, 2)
plt.plot(n_epochs, accuracies)
plt.xticks(n_epochs)
plt.title("Accuracy vs. Number of epochs")
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.show()

As you may see in the plots, the **loss value** decreases as the number of epochs increases. The loss value is a metric that tells you how far from the actual values the model's predictions are. This value and they way it is calculated are directly related to the **loss function**, in this example you used the *Cross Entropy* loss function. 

On the other hand, the **accuracy** represents the quantity of correct predictions out of the total number of samples (in the test dataset). With this model, in the specific iteration when preparing this workshop, the maxmimum obtained value was 70.6%. This means that if you use this model for any medical or diagnostic application you would detect an enhancer correctly 70% percent of the time. Do you think this accuracy is enough?

### Confusion matrix

Another way of assessing how well your model works is with a **confusion matrix**. This is a metric that quantifies and shows the number of correct predictions against the incorrect ones. It compares the number of predicted values against the true ones, allowing to check how often the model misses to clasify correctly a sample.

To calculate it you just need to generate predictions with your model and compare them against the labels contained in the dataset. In the next code cell it's showed how to do this.

In [69]:
model.to(device) # Move the model to the device
model.eval() # Set the model to evaluation mode
size = len(test_loader.dataset)
test_loss, correct = 0, 0
y_pred = [] # Create a list to store the predictions

with torch.no_grad(): # Disable gradient calculation
    for x, y in test_loader:
        x, y = x.to(device), y.to(device) # Move the data to the device
        x = x.unsqueeze(1)
        outputs = model(x) # Get the outputs of the model
        correct += (outputs.round() == y).float().mean() # Get the number of correct predictions
        y_pred.extend(outputs.round().to('cpu').numpy())

In [70]:
# Turn the y_pred values into the same format as the labels contained in the dataset
y_pred = [item.astype('int64').item() for item in y_pred]

In [ ]:
# Calculate confusion matrix 
y_true = test_set.label # Get the true labels
#confusion_matrix(y_true, y_pred) # Calculate the confusion matrix

# Load the confusion matrix metric from HF
confusion_matrix_metric = evaluate.load("confusion_matrix")
# Compute the confusion matrix
confusion_matrix = confusion_matrix_metric.compute(predictions=y_pred, references=y_true.tolist())

# Display the confusion matrix
plt.figure(figsize=(8, 6)) 
sns.heatmap(confusion_matrix['confusion_matrix'], annot=True, fmt='d', cmap=sns.light_palette("seagreen", as_cmap=True)) # Create a heatmap of the confusion matrix
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()

### Receiver-Operator Characteristic (ROC) curve
The **ROC curve** plots the true positive rate (TPR) against the false positive rate (FPR). While it’s typically used for binary classification tasks, it can also be adapted for multi-class classification with some adjustments. An ideal ROC curve would rise sharply from the origin to the top-left corner and then extend horizontally to the top-right corner. The greater the area under the curve (AUC), the better the classifier’s performance, with a full AUC representing a perfect classifier.

In the next code cell, it is shown how to generate a ROC curve using the same `y_true` and `y_pred` variables as in the confusion matrix.

In [ ]:
# Plot a ROC curve 
roc_auc = roc_auc_score(y_true.tolist(), y_pred) # Calculate the ROC AUC score
fpr, tpr, _ = roc_curve(y_true.tolist(), y_pred) # Calculate the false positive rate and true positive rate
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') # Plot the ROC curve
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--') 
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

With the help of the ROC curve it is more clear that the classifier is not ideal for inference in a medical or biological contexts as it would misclassify new and unseen samples.

# Conclusion
In this workshop, you implemented two different deep learning models with varying architectures and complexities to classify genomic sequences. The first model was retrieved from Hugging Face’s Hub, featuring a complex and robust network that was pre-trained, with many helper functions and ready-to-use components, including pre-processed data for training. In the second half of the tutorial, you trained a simple, manually built convolutional neural network. You learned how to construct the essential objects required to train a PyTorch neural network, including `Dataset` and `DataLoader` objects, as well as the actual model.

Since this workshop focused on demonstrating the implementation workflow of these techniques, much of the theory was not included in this Jupyter Notebook. Additionally, the models implemented were relatively simple, resulting in low accuracies and high loss values. In practice, deep learning models used in genomics and other biological fields tend to be far more intricate and complex.

# Contact
## Moises Sotelo Rodriguez

I’m a PhD student specializing in Computational Intelligence at the University Center for Exact Sciences and Engineering, University of Guadalajara. I trained as a Biomedical Engineer and completed my Master’s Degree in Bioengineering and Intelligent Computing. I have also participated in academic stays and a research internship at the Free University of Berlin.

For any inquiries, my academic email is moises.sotelo5881@alumnos.udg.mx, and I would be happy to hear from anyone in the audience today.
### My LinkedIn
<figure>
<img    src="images/msr_linkedin.png"
        width=400
        height=400>
</figure>


## Biodigital Innovation Lab
### At the Biodigital Innovation Lab, we conduct a wide range of experiments, from data analysis and data science to the development of machine and deep learning models. We also perform other types of analyses, such as differential expression, protein folding and hallucination, and the evaluation of their interactions with other molecules of pharmaceutical interest. Our aim is to develop technology and studies that address increasingly ambitious questions.
<figure>
<img    src="images/Logo BIL con nombre.png"
        width=400
        height=500>
</figure>

<figure>
<img    src="images/BIL.jpeg"
        width=900
        height=600>
</figure>

## Our LinkedIn
<figure>
<img    src="images/Linkedin_BIL.png"
        width=400
        height=400>
</figure>

## Our Instagram
<figure>
<img    src="images/IG_QR.jpeg"
        width=400
        height=400>
</figure>

# References

1. Zhang, A., Lipton, Z. C., Li, M., & Smola, A. J. (2021). Dive into deep learning. arXiv preprint arXiv:2106.11342.
2. Avsec, Ž., Agarwal, V., Visentin, D., Ledsam, J. R., Grabska-Barwinska, A., Taylor, K. R., ... & Kelley, D. R. (2021). Effective gene expression prediction from sequence by integrating long-range interactions. Nature methods, 18(10), 1196-1203.
3. Nguyen, E., Poli, M., Faizi, M., Thomas, A., Wornow, M., Birch-Sykes, C., ... & Baccus, S. (2024). Hyenadna: Long-range genomic sequence modeling at single nucleotide resolution. Advances in neural information processing systems, 36.
4. Grešová, K., Martinek, V., Čechák, D., Šimeček, P., & Alexiou, P. (2023). Genomic benchmarks: a collection of datasets for genomic sequence classification. BMC Genomic Data, 24(1), 25. [Genomic Benchmarks paper and repository](https://doi.org/10.1186/s12863-023-01123-8)
5. Clarke, B., Holtkamp, E., Öztürk, H., Mück, M., Wahlberg, M., Meyer, K., ... & Stegle, O. (2024). Integration of variant annotations using deep set networks boosts rare variant association testing. Nature Genetics, 1-10.
6. Zou, J., Huss, M., Abid, A., Mohammadi, P., Torkamani, A., & Telenti, A. (2019). A primer on deep learning in genomics. Nature genetics, 51(1), 12-18.
7. Wolf, T., Debut, L., Sanh, V., Chaumond, J., Delangue, C., Moi, A., ... & Rush, A. M. (2020, October). Transformers: State-of-the-art natural language processing. In Proceedings of the 2020 conference on empirical methods in natural language processing: system demonstrations (pp. 38-45).
8. Erickson, B. J., & Kitamura, F. (2021). Magician’s corner: 9. Performance metrics for machine learning models. Radiology: Artificial Intelligence, 3(3), e200126. https://doi.org/10.1148/ryai.2021200126
9. Harrison, P. W., Amode, M. R., Austine-Orimoloye, O., Azov, A. G., Barba, M., Barnes, I., Becker, A., Bennett, R., Berry, A., Bhai, J., Bhurji, S. K., Boddu, S., Branco Lins, P. R., Brooks, L., Ramaraju, S. B., Campbell, L. I., Martinez, M. C., Charkhchi, M., Chougule, K., … Yates, A. D. (2024). Ensembl 2024. Nucleic Acids Research, 52(D1), D891–D899. https://doi.org/10.1093/nar/gkad1049
